In [ ]:
from twilio.rest import Client
import pandas as pd 
import numpy as np 
import plotly.express as px
import psutil # CPU info
import plotly.express as px
import multiprocessing as mp
import plotly.graph_objects as go
import time 
import glob
import shutil, os
import json
from re import search
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from twilio.rest import Client
from datetime import datetime
import sys
from distutils.dir_util import copy_tree
import plotly


In [ ]:
# Add a column to df seperating date and time 1900-01-01 TimeStamp = 11:39:09.625
def split_date_time(hr):
    ts_only = []
    for i in hr["Time"]:
        ts_only.append(i[11:])
    hr["TimeStamp"] = ts_only
    
# Add a column to the dataframe containing 12 hour timestamp.
def convert_ts_human(hr):
    # Convert all timetamps to 12 hour format
    conveted_human_list = []
    for i in hr["Time"]:
        d = datetime.strptime(i.split(" ")[1], "%H:%M:%S.%f")
        conveted_human_list.append(d.strftime("%I:%M:%S %p")) #12 HR FORMAT.
# To filter by Hour and Date&Time. Give it a dataset for one day only apply that filter before feeding.

def unique_time_stamps(df):
    time_stamp_list = [] # get the unique time stamps int he paricular df.
    for idx,val in enumerate(df["Time"]):
        #print("TS: ",val,val[11:13])
        time_stamp_list.append(val[11:13])
    return np.unique(np.array(time_stamp_list))

In [ ]:
p_27 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
p_34 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
p_52 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
p_53 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
p_75 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
p_80 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
p_88 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p_90 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
p_106 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
p_118 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
p_129 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
p_131 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
# Returns a list of Dates based on Date ID.
def get_dates(patient_id):
    date_id_list = []
    if patient_id == 27:
        date_id_list = p_27
    elif patient_id == 34:
        date_id_list = p_34
    elif patient_id == 52:
        date_id_list = p_52
    elif patient_id == 53:
        date_id_list = p_53
    elif patient_id == 75:
        date_id_list = p_75
    elif patient_id == 80:
        date_id_list = p_80
    elif patient_id == 88:
        date_id_list = p_88
    elif patient_id == 90:
        date_id_list = p_90
    elif patient_id == 106:
        date_id_list = p_106
    elif patient_id == 118:
        date_id_list = p_118
    elif patient_id == 129:
        date_id_list = p_129
    elif patient_id == 131:
        date_id_list = p_131
    return date_id_list

In [ ]:
# Returns a list of time in secods per activity bucket.
def calculate_time_seconds_each_act(hr):
    total_time_each_bucket_seconds = []
    unique_ts_each_bucket = []
    print("Number of buckets ",hr["HR_ACT_ZONE_MORE"].unique())
    bucket_info_list = hr["HR_ACT_ZONE_MORE"].unique()
    for bucket in [0,1,2,-1]:
        print("For BUCKET",bucket)
        # Both should have same number of elements.
        start_ts_hr_list = []
        end_ts_hr_list = []
        time_seconds_list = []
        #print("Bucket",bucket,"Data shape", hr[hr["HR_ACT_ZONE_MORE"]==bucket].shape[0])
        #print("Data = ", hr[hr["HR_ACT_ZONE_MORE"]==bucket])
        unique_ts_beyond_act = unique_time_stamps(hr[hr["HR_ACT_ZONE_MORE"]==bucket])
        unique_ts_each_bucket.append(unique_ts_beyond_act)
        print("UNIQUE TS LIST  = ",len(unique_ts_beyond_act))
        for ts_hour in unique_ts_beyond_act:
            # In order to calculate time we bucket the timestamp by hour and then for each hour find start and end times and take the diff between those and them sum everything.
            val_bool = []
            val = []
            for i in hr[hr["HR_ACT_ZONE_MORE"]==bucket]["TimeStamp"]:
                # Bool Ture if condition is met.
                val_bool.append(i.startswith(ts_hour))
                val.append(i) # Also create a list of actual values.
            # Convert to pandas aeries in order to use where.
            val_series = pd.Series(val_bool)
            start_index = val_series.where(val_series==True).first_valid_index()
            end_index = val_series.where(val_series==True).last_valid_index()
            start_ts = datetime.strptime(val[start_index], "%H:%M:%S.%f") 
            end_ts = datetime.strptime(val[end_index], "%H:%M:%S.%f")
            time_seconds = (end_ts - start_ts).total_seconds()
            start_ts_hr_list.append(start_ts)
            end_ts_hr_list.append(end_ts)
            #print("Each time in seconds",time_seconds)
            time_seconds_list.append(time_seconds/60) # Length of this list will vary based on ts hour info avaliable.
        print("time_seconds_list Time in SECONDS list = ",time_seconds_list,"Length: ",len(time_seconds_list))
        #time_seconds_list
        total_time = sum(time_seconds_list)
        print("Total Time = ",total_time)
        total_time_each_bucket_seconds.append(total_time)
    # Converted text for plot annotations.
    str_text_time_mins = []
    for i in total_time_each_bucket_seconds:
        str_text_time_mins.append(str(int(i)) + " Minutes")
    return total_time_each_bucket_seconds,str_text_time_mins,bucket_info_list


### Calculate Activity Level Stats: 

In [ ]:
#patient_id = 88
patient_id_list = [27,34,52,53,75,80,88,90,106,118,129,131,584]
for patient_id in patient_id_list:
    print("--------Patient ID -------- :", patient_id)
    start_time = time.time()
    date_id_list = get_dates(patient_id)
    #print("date_id_list = ", date_id_list)
    day_wise_mins = []
    day_wise_mins_annotation_fotmat = []
    bucket_info_day_list = []
    # For all Dates of a particlar Patient.
    for i in date_id_list:
        print("--------Date-------- :", i)
        path = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/ResearchWBL/My_Thesis/careportal-flaskapp/care-portal-thesis-sub/PatientID_" +str(patient_id) + "/HR_ACT_Resample_40Hz_PID_" + str(patient_id) + "_" + str(i) + "_Date_ID_HR.csv"
        hr = pd.read_csv(path)
        split_date_time(hr)
        total_time_each_bucket_mins,str_text_time_mins,bucket_info_list = calculate_time_seconds_each_act(hr)
        day_wise_mins.append(total_time_each_bucket_mins)
        day_wise_mins_annotation_fotmat.append(str_text_time_mins)
        bucket_info_day_list.append(bucket_info_list)
    stop_time = time.time()
    day_pandas = pd.DataFrame(day_wise_mins,columns=["LessIntense","ModerateIntensity","HighIntensity","BeyondActivity"])
    day_pandas["Dates"] = date_id_list
    annotation_df = pd.DataFrame(day_wise_mins_annotation_fotmat,columns=["LessIntense","ModerateIntensity","HighIntensity","BeyondActivity"])
    json_Act_levels = {}
    for i in day_pandas["Dates"]:
        json_Act_levels[i] = day_pandas.iloc[i].to_list()
    r = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/ResearchWBL/My_Thesis/careportal-webapp/static/plotlycharts/"
#     with open(r+"PatientID_"+ str(patient_id)+"timming_"+str(patient_id)+"activity_levels.json", 'w') as outfile:
#         json.dump(json_Act_levels, outfile)



In [ ]:
annotation_df

In [ ]:
patient_id =88
r = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/ResearchWBL/My_Thesis/careportal-webapp/static/plotlycharts/"
with open(r+"PatientID_"+ str(patient_id)+"/plotly_hr_zone_"+str(patient_id)+"activity_levels.json", 'w') as outfile:
    json.dump(json_Act_levels, outfile)
    

In [ ]:
path = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/ResearchWBL/My_Thesis/careportal-webapp/static/plotlycharts/PatientID_88/PatientID_88timming_88activity_levels.json"
# act_levels = json.load(path)
# read file
with open(path, 'r') as myfile:
    act_levels=myfile.read()


In [ ]:
act_levels_json = json.loads(act_levels)
dates = get_dates(patient_id)

In [ ]:
less_intense = []
moderate_intense = []
high_intensity = [] 
beyond_activity = []
sum_list = []
for i in act_levels_json.keys():
    #print(act_levels_json[i][0])
    less_intense.append(act_levels_json[i][0])
    moderate_intense.append(act_levels_json[i][1])
    high_intensity.append(act_levels_json[i][2])
    beyond_activity.append(act_levels_json[i][3])
    


In [ ]:
totals_day_list = []
totals_no_act_list = []
for less_intense1, moderate_intense1,high_intensity1,beyond_activity1 in zip(less_intense, moderate_intense,high_intensity,beyond_activity):
    s = less_intense1 + moderate_intense1 + high_intensity1 +beyond_activity1
    totals_day_list.append(s)
    left_over = 1440 - s
    print("Act",s,"Left Over",left_over,"Total",left_over+s)
    totals_no_act_list.append(left_over)
    

In [ ]:
widths = []
for i in range(0,len(dd[0])):
    widths.append(10)


In [ ]:
widths_np = np.array(widths)

In [ ]:
dd = [less_intense,moderate_intense,high_intensity  , beyond_activity]


# SAME HEIGHT FOR BAR PLOTS (Viz no data points)

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='No Data', x=dates, y=totals_no_act_list[:7],marker_color='lightslategrey'),
    go.Bar(name='No Activity', x=dates, y=less_intense[:7],marker_color = "teal"),
    go.Bar(name='Moderate Activity', x=dates, y=moderate_intense[:7],text=moderate_intense,marker_color ="darkseagreen"),
    go.Bar(name='High Activity', x =dates, y=high_intensity[:7],text=high_intensity,marker_color ="lightseagreen"),
    go.Bar(name='Beyond Activity',x=dates, y=beyond_activity[:7],text=beyond_activity,marker_color ="red")])
fig.update_xaxes(title="Day")
fig.update_yaxes(title="Number of Mins")
fig.update_layout(barmode='stack',title ="Cardiac Activity Levels")
fig.show()


In [ ]:
dates = []
for i in act_levels_json.keys():
    dates.append(i)

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='No Activity', x=dates, y=less_intense),
    go.Bar(name='Moderate Activity', x=dates, y=moderate_intense,text=moderate_intense),
    go.Bar(name='High Activity', x=dates, y=high_intensity,text=high_intensity),
    go.Bar(name='Beyond Activity', x=dates, y=beyond_activity,text=beyond_activity)
])

fig.update_xaxes(title="Day")
fig.update_yaxes(title="Activity in Mins")
fig.update_layout(barmode='stack',title ="Cardiac Activity Levels")
fig.show()
# Can improve this chart by makeing the bar stay static.
